In [ ]:
# import os, datetime
# import github
# from progress.bar import ShadyBar
# from progress.spinner import Spinner
# import secrets
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from flask import Flask
from model import (Repo, User, Follower,
                   Stargazer, Watcher, Contributor,
                   Language, RepoLanguage,
                   db, connect_to_db, db_uri)
import utils

In [ ]:
app = Flask(__name__)
connect_to_db(app)

In [ ]:
# users = User.query.all()
# users_list = [[user.user_id, user.login, user.name]  for user in users]

# repos = Repo.query.all()
# repos_list = [[repo.repo_id, repo.name] for repo in repos]
# repos_df = pd.DataFrame(repos_list, columns = ["repo_id", "repo_name"])

ratings = Stargazer.query.all()
ratings_list = [[rating.user_id, rating.repo_id, 1] for rating in ratings]
ratings_df = pd.DataFrame(ratings_list, columns = ["user_id", "repo_id", "Rating"], dtype = int)
R_df = ratings_df.pivot(index="user_id", columns = "repo_id", values = "Rating").fillna(0)
R = R_df.as_matrix()

In [ ]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [ ]:
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
predictions_df = pd.DataFrame(predicted_ratings, columns = R_df.columns, index = R_df.index)
predictions_df[predictions_df < 0.99].loc[10337420].sort_values(ascending=False).index.tolist()

In [ ]:
# print(user_ratings_mean)
# for mean in user_ratings_mean:
#     print(mean)
# print(R_demeaned)
# print(sigma)
# predictions_df.head()
user_ratings_mean_df = pd.DataFrame(user_ratings_mean, index = R_df.index)
# user_ratings_mean_df.sort_values(ascending=False)

In [ ]:
user_ratings_mean_df.iloc[:,0].sort_values(ascending=False).head()

In [ ]:
# predictions_df[predictions_df < 1].iloc[77].sort_values(ascending=False)
predictions_df.iloc[77].sort_values(ascending=False).head()

In [ ]:
predictions_df.head()

In [ ]:
User.query.get(10337420)

In [ ]:
predictions_df[predictions_df < .99].loc[10337420].sort_values(ascending=False).head()

In [ ]:
predictions_df[predictions_df < 0.99].loc[10337420].sort_values(ascending=False).index.tolist()